In [2]:
import json
from urllib.request import urlopen
from pprint import pprint
from copy import deepcopy
import requests

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
#the initially url requesting and json opening
biblio = "http://adsorbents.nist.gov/api/biblio.json"
webrequest = urlopen(biblio)
loadwebrequest = json.load(webrequest)

In [1]:
# #commented out because it turns out the vast majority of papers did not have author affiliation?? recheck later
# doilist = []
# for dic in loadwebrequest:
#     for key, doi in dic.items():
#         if key == "DOI":
#             doilist.append(doi)
# doilist = list(enumerate(doilist))

# for x in doilist: 
#     DOI = x[1]

#     try:
#         url = 'http://dx.doi.org/'+DOI
#         headers = {'Accept': 'application/citeproc+json'} # JSON Headers
#         bib_info = json.loads(requests.get(url, headers=headers).content)
#         pprint(bib_info)
#     except:
#         raise RuntimeError("ERROR: DOI problem for : "+DOI)
# for dic in bib_info:
#     for key, affiliation in dic:
#         if key == "affiliation":
#             if affiliation != []:
#                 print("yas")
# pprint(loadwebrequest)

In [6]:
#establishes some lists and parses for author names. also indexes the names for connecting back to their original papers
preoriginallist = []
comparisonlist = []

for dic in loadwebrequest:
    for key, name in dic.items():
        if key == "authors":
            preoriginallist.append(name)
preoriginallist = list(enumerate(preoriginallist))


In [7]:
#removes the index (index[0]) in the first nested loop, places the index after each name
originallist = deepcopy(preoriginallist)

for b in range(len(originallist)):
    for g in range(1, len(preoriginallist[b][1])*2,2):
        originallist[b][1].insert(g, preoriginallist[b][0]) 
originallist = [list(g) for g in originallist]
for k in originallist:
    k.pop(0)
    
#pprint(originallist)

In [8]:
#splits the names into lists of each name (e.g. first, last) for comparisons, also creates a new list composing of 
#names that have "." in it (AKA initials) to be matched
originalsplitlist = []
neworiginalsplitlist = []
comparisonsplitlist = []
for uncomplete in originallist:
    for blah in uncomplete:
        for flah in blah:
            originalsplitlist.append(str((flah)).split())
for q in originalsplitlist:
    for p in originallist:
        for n in range(0, int(len(p)/2)):
            q.insert(0,originallist[n]*3)
for a in range(int(len(originalsplitlist)/2)):
    neworiginalsplitlist.insert(0, (originalsplitlist[a*2] + originalsplitlist[a*2+1]))
for v in neworiginalsplitlist:
    for minilist in v:
        for stringletter in minilist:
            if stringletter == ".":
                comparisonsplitlist.append(v)
print(len(comparisonsplitlist))

5114


In [9]:
# totalauthorcount = 0
# for x in originallist:
#     for y in x:
#         for z in y:
#             if type(z) == str:
#                 totalauthorcount = totalauthorcount + 1
# print(totalauthorcount)
print(len(comparisonsplitlist))

5114


In [19]:
#sorts the names according to having the same last name, same first initial, same length, and not being identical and matches
possiblematches = []
for ogsplitname in neworiginalsplitlist:
    for splitname in comparisonsplitlist:
        if ogsplitname[-2] == splitname[-2] and ogsplitname[0][0] == splitname[0][0] and ogsplitname != splitname and len(ogsplitname) == len(splitname) and ogsplitname[0] != splitname[0]:
            if len(ogsplitname) == 4:
                if ogsplitname[1][0] == splitname[1][0]:
                    possiblematches.append(splitname + ogsplitname)
            else: 
                possiblematches.append(splitname + ogsplitname)

possiblematches = list(tuple((set(tuple(x) for x in possiblematches))))
possiblematches = [list(x) for x in possiblematches]

print(len(possiblematches))

2726


In [11]:
DOIlist = []
for dic in loadwebrequest:
    for key, doi in dic.items():
        if key == "DOI":
            DOIlist.append(doi)
DOIlist = list(enumerate(DOIlist))
# pprint(DOIlist)

[(0, '10.1002/adem.200500223'),
 (1, '10.1002/adem.201000246'),
 (2, '10.1002/adfm.200500561'),
 (3, '10.1002/adfm.200500830'),
 (4, '10.1002/adfm.200600944'),
 (5, '10.1002/adfm.200800058'),
 (6, '10.1002/adfm.200900880'),
 (7, '10.1002/adfm.201002517'),
 (8, '10.1002/adfm.201101479'),
 (9, '10.1002/adfm.201200084'),
 (10, '10.1002/adfm.201300071'),
 (11, '10.1002/adfm.201300510'),
 (12, '10.1002/adfm.201301178'),
 (13, '10.1002/adfm.201302854'),
 (14, '10.1002/adfm.201400559'),
 (15, '10.1002/adfm.201502069'),
 (16, '10.1002/adma.200400946'),
 (17, '10.1002/adma.200601983'),
 (18, '10.1002/adma.200602645'),
 (19, '10.1002/adma.200700450'),
 (20, '10.1002/adma.200702605'),
 (21, '10.1002/adma.200802485'),
 (22, '10.1002/adma.200901581'),
 (23, '10.1002/adma.201102116'),
 (24, '10.1002/adma.201102538'),
 (25, '10.1002/adma.201203383'),
 (26, '10.1002/adma.201203664'),
 (27, '10.1002/adma.201400428'),
 (28, '10.1002/adma.201403827'),
 (29, '10.1002/adma.201500966'),
 (30, '10.1002/adma.

In [12]:
#uses the matches from the previous block: goes back to their papers in originallist to compare the # of similar authors
plausiblematches = deepcopy(possiblematches)
for x in range(len(possiblematches)):
    authormatchcount = 0
    for y in originallist:
        for j in y:
            if int(possiblematches[x][len(possiblematches[x])-1]) in j:
                for z in originallist:
                    for k in z:
                        if int(possiblematches[x][int(len(possiblematches[x])/2-1)])in k:
                            for author1 in j:
                                if author1 in k:
                                    authormatchcount = authormatchcount + 1 
    plausiblematches[x].append(str(authormatchcount)) #so last number == # of author matches
pprint(sorted(plausiblematches, key=lambda match: match[-1], reverse=True))

[['N.', 'Rosenbach', '1883', 'Nilton', 'Rosenbach', '1857', '9'],
 ['F.', 'Ragon', '109', 'Florence', 'Ragon', '2996', '8'],
 ['Q.', 'Yang', '109', 'Qingyuan', 'Yang', '2996', '8'],
 ['J.', 'Yang', '730', 'Jun', 'Yang', '1074', '7'],
 ['C.', 'Xu', '730', 'Chunchuan', 'Xu', '1074', '7'],
 ['Q.', 'Yang', '2030', 'Qingyuan', 'Yang', '2031', '7'],
 ['N.', 'Rosenbach', '1883', 'Nilton', 'Rosenbach', '76', '6'],
 ['H.', 'Jobic', '3190', 'Hervé', 'Jobic', '2330', '6'],
 ['H.', 'Jobic', '3190', 'Hervé', 'Jobic', '2114', '6'],
 ['C.', 'Xu', '730', 'Chunchuan', 'Xu', '1293', '6'],
 ['H.', 'Jobic', '1026', 'Hervé', 'Jobic', '2067', '6'],
 ['H.', 'Jobic', '2102', 'Hervé', 'Jobic', '2067', '6'],
 ['H.', 'Jobic', '3190', 'Hervé', 'Jobic', '2164', '6'],
 ['J.', 'Yang', '730', 'Jun', 'Yang', '1293', '6'],
 ['P.', 'Ungerer', '357', 'Philippe', 'Ungerer', '3268', '6'],
 ['H.', 'Jobic', '3190', 'Hervé', 'Jobic', '2024', '6'],
 ['J.', 'Yang', '730', 'Jun', 'Yang', '2075', '6'],
 ['H.', 'Jobic', '3190', 'H

In [14]:
plausiblematcheswithdoi = deepcopy(plausiblematches)
for x in DOIlist:
    for y in range(len(possiblematches)):
#         print(len(possiblematches[y]))
#         print(str(x[0]) == y[2 + (len(possiblematches[y]) - 6)])
#         print(str(x[0]))
#         print(y[2+(6-len(possiblematches[y]))])
        if str(x[0]) == possiblematches[y][2 + int(.5*(len(possiblematches[y]) -6))]:
            plausiblematcheswithdoi[y].insert(3+ int(.5*(len(possiblematches[y]) -6)), x[1])
        elif str(x[0]) == possiblematches[y][-1]:
            plausiblematcheswithdoi[y].insert(-1, x[1])
        
plausiblematcheswithdoi = sorted(plausiblematcheswithdoi, key=lambda match: match[-1], reverse=True)
for x in plausiblematcheswithdoi:
    print(x)
# pprint(possiblematches)
#2726 total matches

['N.', 'Rosenbach', '1883', '10.1021/Ja902740r', 'Nilton', 'Rosenbach', '1857', '10.1021/Ja803899q', '9']
['F.', 'Ragon', '109', '10.1002/anie.201204806', 'Florence', 'Ragon', '2996', '10.1039/c4ta03992k', '8']
['Q.', 'Yang', '109', '10.1002/anie.201204806', 'Qingyuan', 'Yang', '2996', '10.1039/c4ta03992k', '8']
['J.', 'Yang', '730', '10.1016/j.ijhydene.2012.12.096', 'Jun', 'Yang', '1074', '10.1016/j.micromeso.2013.11.015', '7']
['C.', 'Xu', '730', '10.1016/j.ijhydene.2012.12.096', 'Chunchuan', 'Xu', '1074', '10.1016/j.micromeso.2013.11.015', '7']
['Q.', 'Yang', '2030', '10.1021/Jp2039527', 'Qingyuan', 'Yang', '2031', '10.1021/Jp205369t', '7']
['N.', 'Rosenbach', '1883', '10.1021/Ja902740r', 'Nilton', 'Rosenbach', '76', '10.1002/anie.200801748', '6']
['H.', 'Jobic', '3190', '10.1103/Physrevlett.100.245901', 'Hervé', 'Jobic', '2330', '10.1021/Nn901132k', '6']
['H.', 'Jobic', '3190', '10.1103/Physrevlett.100.245901', 'Hervé', 'Jobic', '2114', '10.1021/Jp403225t', '6']
['C.', 'Xu', '730',

In [15]:
# for x in plausiblematches:
#     if x[-1] != "0 author matches":
#         #pprint(x)
# #538 total entries
# print(DOIlist[1857])

(1857, '10.1021/Ja803899q')


In [16]:
# journallist = []
# for dic in loadwebrequest:
#     for key, journalname in dic.items():
#         if key == "journal":
#             journallist.append(journalname)
# journallist = list(enumerate(journallist))
# pprint(journallist[2645])
# #333 different journals

(2645, 'Chemical Communications')


In [17]:
# plausiblejournalmatches = deepcopy(plausiblematches)
# for i in plausiblejournalmatches:
# #     print(i[6])
#     for j in journallist:
#         if str(j[0]) == i[2]:
# #                 print(j[0])
# #                 print(i[2])
#             i.insert(0, j[1])
# #                 print(i[0])
#         if str(j[0]) == i[6] or str(j[0]) == i[5]:
#             i.insert(-1, j[1])
#         #print(str(j[0]) == i[6] or str(j[0]) == i[5])
#     #print(i[3])
# pprint(plausiblejournalmatches)


In [18]:
# possiblematcheswithjournals = []
# for i in journallist:
#     for j in possiblematches:
#         if i[0] == j[0]:
#             possiblesmatcheswithjournals.append(i[0] + j[0])
#         elif i[0] == j